In [1]:
import numpy as np
import time
import sys
import os
import tqdm

import torch
from torch import nn
import torch.nn.functional as F


from catr.configuration import Config
from catr.models.utils import NestedTensor, nested_tensor_from_tensor_list, get_rank
from catr.models.backbone import build_backbone
from catr.models.transformer import build_transformer
from catr.models.position_encoding import PositionEmbeddingSine
from catr.models.caption import MLP

import json

from dataset.dataset import ImageFeatureDataset
from torch.utils.data import DataLoader
from transformer_ethan import *
sys.path.append(os.path.join(os.path.dirname("__file__"), "catr"))
from engine import train_one_epoch, evaluate

In [2]:
words = np.load("glove_embed.npy")
with open('word2ind.json') as json_file: 
    word2ind = json.load(json_file) 
with open('ind2word.json') as json_file: 
    ind2word = json.load(json_file) 
config = Config()
# config.device = 'cpu' # if running without GPU
config.feature_dim = 1024
config.pad_token_id = word2ind["<S>"]
config.hidden_dim = 300
config.nheads = 10
config.batch_size = 32
config.encoder_type = 1
config.vocab_size = words.shape[0]
config.dir = '../mimic_features_double'
config.__dict__["pre_embed"] = torch.from_numpy(words).to(config.device)

In [3]:
model, criterion = main(config)
model = model.float()
device = torch.device(config.device)
model.to(device)

Initializing Device: cuda
Number of params: 33370520


Xray_Captioner(
  (input_proj): Conv2d(1024, 300, kernel_size=(1, 1), stride=(1, 1))
  (position_embedding): PositionEmbeddingSine()
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=300, out_features=300, bias=True)
          )
          (linear1): Linear(in_features=300, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=300, bias=True)
          (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=300, out_features=3

In [4]:
param_dicts = [
        {"params": [p for n, p in model.named_parameters(
        ) if "backbone" not in n and p.requires_grad]},
        {
            "params": [p for n, p in model.named_parameters() if "backbone" in n and p.requires_grad],
            "lr": config.lr_backbone,
        },
    ]

In [5]:
optimizer = torch.optim.AdamW(
        param_dicts, lr=config.lr, weight_decay=config.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, config.lr_drop)

In [6]:
dataset_train = ImageFeatureDataset(config, mode='train')
dataset_val = ImageFeatureDataset(config, mode='val')

sampler_train = torch.utils.data.RandomSampler(dataset_train)
sampler_val = torch.utils.data.SequentialSampler(dataset_val)

batch_sampler_train = torch.utils.data.BatchSampler(
        sampler_train, config.batch_size, drop_last=True)

data_loader_train = DataLoader(
        dataset_train, batch_sampler=batch_sampler_train, num_workers=config.num_workers)
data_loader_val = DataLoader(dataset_val, config.batch_size,
                                 sampler=sampler_val, drop_last=False, num_workers=config.num_workers)
print(f"Train: {len(dataset_train)}")
print(f"Val: {len(dataset_val)}")

Train: 253893
Val: 1196


In [7]:
if os.path.exists("./checkpoint_imagenet_10_tf.pth"):
      print("Loading Checkpoint...")
      checkpoint = torch.load("./checkpoint_imagenet_10_tf.pth")#, map_location='cuda')
      model.load_state_dict(checkpoint['model'])
      optimizer.load_state_dict(checkpoint['optimizer'])
      lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
      config.start_epoch = checkpoint['epoch'] + 1

print("Start Training..")

Loading Checkpoint...
Start Training..


In [ ]:
train_loss_hist = []
val_loss_hist = []
train_bleu_hist = []
val_bleu_hist = []

for epoch in range(config.start_epoch, config.epochs):
    print(f"Epoch: {epoch}")
    epoch_loss, train_bleu_score = train_one_epoch(
        model, criterion, data_loader_train, optimizer, device, epoch, config.clip_max_norm, word2ind)
    train_loss_hist.append(epoch_loss)
    train_bleu_hist.append(train_bleu_score)
    lr_scheduler.step()
    print(f"Training Bleu Score: {train_bleu_score}")
    print(f"Training Loss: {epoch_loss}")

    torch.save({
         'model': model.state_dict(),
         'optimizer': optimizer.state_dict(),
         'lr_scheduler': lr_scheduler.state_dict(),
         'epoch': epoch,
     }, config.checkpoint)

    validation_loss, val_bleu_score = evaluate(model, criterion, data_loader_val, device, word2ind)
    val_loss_hist.append(validation_loss)
    val_bleu_hist.append(val_bleu_score)
    print(f"Validation Bleu Score: {val_bleu_score}")
    print(f"Validation Loss: {validation_loss}")

    print()

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Epoch')
plt.ylabel('Val Bleu')
# plt.grid()
plt.plot(val_bleu_hist)

# Evaluation helper funcs

In [8]:
# Edward: note this makes a new caption as (<S>, 0, ..., 0) shouldn't we want as (<S>, <S>, ..., <S>)?
def create_caption_and_mask(start_token, max_length):
    caption_template = torch.zeros((1, max_length), dtype=torch.long)
    mask_template = torch.ones((1, max_length), dtype=torch.bool)

    caption_template[:, 0] = start_token
    mask_template[:, 0] = False

    return caption_template, mask_template

In [9]:
def make_report(captions):
    all_reports = []
    for report in captions:
        if (report == word2ind["</s>"]).any():
            end_index = (report == word2ind["</s>"]).nonzero()[0][0]
            report = report[:end_index+1]
        one_report = list(map(lambda x: ind2word[str(x)], report))
        all_reports.append(one_report)
    return all_reports

def reports_to_sentence(reports):
    return [' '.join(r) for r in make_report(reports)]

In [10]:
def evaluate(images):
    all_captions = []
    model.eval()
    for i in range(len(images)):
        image = images[i:i+1]
        caption, cap_mask = create_caption_and_mask(
            config.pad_token_id, config.max_position_embeddings)
        caption.to("cuda")
        for i in range(config.max_position_embeddings - 1):
            predictions = model(image, caption, cap_mask).to(config.device)
            predictions = predictions[:, i, :]
            predicted_id = torch.argmax(predictions, axis=-1)


            caption[:, i+1] = predicted_id[0]
            cap_mask[:, i+1] = False
            
            if predicted_id[0] == word2ind["</s>"]:
                break

        all_captions.append(caption.numpy())
#     return make_report(all_captions)
    return all_captions

In [11]:
import nltk
sample_bleu4 = []
sample_bleu3 = []
sample_bleu2 = []
sample_bleu1 = []

In [12]:
iterations = iter(data_loader_val)

In [37]:
image, note, note_mask = next(iterations)

In [38]:
report = evaluate(image.to("cuda"))

In [39]:
report_np = np.asarray(report).squeeze(1)
#reports_to_sentence(report_np)

In [40]:
#reports_to_sentence(np.asarray(note[:,:]))

In [41]:
for index in range(config.batch_size):
    truth = reports_to_sentence(np.asarray(note[:,:]))[index]
    generated = reports_to_sentence(report_np)[index]
    truth = truth.replace("<S>", "").replace("<s>", "").replace("</s>", "").replace(".", "").replace(",", "").replace("  ", " ").split(" ")
    generated = generated.replace("<S>", "").replace("<s>", "").replace("</s>", "").replace(".", "").replace(",", "").replace("  ", " ").split(" ")
    truth = [y for y in truth if y != ''] 
    generated = [y for y in generated if y != ''] 
    bs4 = nltk.translate.bleu_score.sentence_bleu([truth], generated, weights=[0.25, 0.25, 0.25, 0.25])
    bs3 = nltk.translate.bleu_score.sentence_bleu([truth], generated, weights=[1./3., 1./3., 1./3.])
    bs2 = nltk.translate.bleu_score.sentence_bleu([truth], generated, weights=[0.5, 0.5])
    bs1 = nltk.translate.bleu_score.sentence_bleu([truth], generated, weights=[1.])
    sample_bleu4.append(bs4)
    sample_bleu3.append(bs3)
    sample_bleu2.append(bs2)
    sample_bleu1.append(bs1)
    print("Bleu score: ", bs1, bs2, bs3, bs4)

Bleu score:  0.14485827874984 0.057672546880514215 3.037995964945171e-104 2.2049371952991427e-155
Bleu score:  0.35546510772836076 0.1797873337323777 0.11458897990426707 0.07739155867370852
Bleu score:  0.2888449441847639 0.16490550987785255 0.10399385051580423 0.06314841251800533
Bleu score:  0.21874999999999997 0.08400268812903088 5.394819679667346e-104 4.3233341924983234e-155
Bleu score:  0.3070848473887668 0.1483360738852236 7.052737524712083e-104 4.863099910606428e-155
Bleu score:  0.39414977871115225 0.230790478678117 0.11401004817750031 2.3632604499229856e-78
Bleu score:  0.0625 0.03646624787447365 3.0929687369589304e-104 2.848510046732156e-155
Bleu score:  0.09004363613933614 0.0478010601522781 0.034196083015794035 0.02453568980870263
Bleu score:  0.06666666666666667 3.851470592303926e-155 3.2077403065267053e-206 9.257324954728539e-232
Bleu score:  0.11725279135918835 0.07924676509472729 0.05571889317867387 0.03957334239380666
Bleu score:  0.09631542180526126 0.0694540383873623

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it cont

In [46]:
sum(sample_bleu4)/len(sample_bleu1)

0.0645436404955807

In [42]:
len(sample_bleu1)

128